<a href="https://colab.research.google.com/github/nassanea/ML2021/blob/main/HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [ ]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
!unzip -q food-11.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
From (redirected): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy&confirm=t&uuid=f5af831a-df61-4aed-910a-77e2256ffec2
To: /content/food-11.zip
100% 963M/963M [00:07<00:00, 127MB/s]


## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms # 用來對影像進行前處理與資料增強
from torchvision.transforms import InterpolationMode
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
# Subset 與 ConcatDataset 通常用在你要自己切資料集或是做半監督學習時才需要用到。
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder


# 匯入 tqdm 的自動模式進度條，在訓練時可以看到 loop 執行進度。
from tqdm.auto import tqdm

## **Dataset, Data Loader, and Transforms**

過 medium base line 的關鍵

**Torchvision provides lots of useful utilities for image preprocessing**, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

***

To.Tensor()：

將 PIL 圖片或 NumPy 陣列 ➜ 轉換成 PyTorch Tensor，並且
1. **資料型態**轉為 torch.FloatTensor（浮點數）
2. 像素值從 [0, 255] 正規化為 [0.0, 1.0]
3. 維度重新排列：
  - 從 (H, W, C) 轉為 (C, H, W)
  - PIL Image：Height × Width × Channels（高×寬×通道）
  - PyTorch Tensor：Channels × Height × Width（通道×高×寬）

=> 因為：
- PyTorch 的模型只接受 Tensor 形式的資料做訓練或推論
- 正規化到 [0, 1] 範圍可以讓神經網路更穩定地學習，較小的數值範圍有助於模型訓練

In [ ]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.

# 訓練資料變換
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # use data argument (add transforms)
    transforms.RandomHorizontalFlip(p=0.5), # 隨機水平翻轉
    transforms.RandomRotation(degrees=20,
                             interpolation=InterpolationMode.BILINEAR, # 隨機旋轉角度，interpolation為決定旋轉時如何填補新像素的顏色
                             expand=False), # 預設，維持圖片原尺寸

    # Jitter -> 讓數據不完美，ColorJitter 讓圖片的顏色相關產生輕微變化
    transforms.ColorJitter(brightness=0.2, # 明亮度
                          contrast=0.2, # 對比度
                          saturation=0.2, # 飽和度
                          hue=0.1 ), # 色相

    transforms.RandomPerspective(distortion_scale=0.2, # 對圖片施加透視投影變形（模擬斜拍、拉伸）
                                p=0.3),

    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# 測試資料變換（不用對測試資料做資料增強）
# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
# 針對訓練集的設定
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]（是 [channel x 長 x 寬] 的 tensor）
        self.cnn_layers = nn.Sequential(
            # 卷積
            nn.Conv2d(3, 64, 3, 1, 1), # -> output shape: [64, 128, 128]
            nn.BatchNorm2d(64), # 批次正規化，要搭配 CNN 的 out_channels = 64
            nn.ReLU(),
            # 沒改變維度但加深網路深度的卷積層
            # 即使輸入輸出維度沒變，每一層卷積還是能抽取 更多不同的空間特徵
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # 池化層
            # 壓縮資訊、減少計算量
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0), # 128 / 2 = 64 -> [64, 64, 64]


            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0), # 64 / 2 = 32 -> [128, 32, 32]


            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0), # 32 / 4 = 8 -> [256, 8, 8]
            # 變成 [channel = 256, 長 = 8, 寬 = 8] 的 tensor


        )
        self.fc_layers = nn.Sequential(
            # 當在 forward 中經由 fatten 攤平成維度 = 256*8*8 的向量後，就可以通過 FC layer
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11) # 輸出 11 類分類結果
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

過 strong base line 的關鍵

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [ ]:
# 定義一個函數來生成偽標籤
def get_pseudo_labels(dataset, model, threshold=0.65):
# dataset ：傳入待加上標籤的資料集(unlabeled_set)；model ：用來預測的模型
# threshold：信心閾值，只有預測信心超過此值的樣本才會被加上偽標籤
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.建立未標記資料的 data loader
    # 前面建立 data loader 時尚未用到 unlabeled_set，故此處用上
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Make sure the model is in eval mode.
    # 設為評估模式，會停用 dropout 和 batchnorm 的隨機行為
    model.eval()

    # Define softmax function.
    # 用來將模型輸出的 logits 轉換為機率分布（在最後一個維度上進行softmax）
    softmax = nn.Softmax(dim=-1) # -1 代表最後一個維度（-2 代表倒數第二個維度，類推）

    # Iterate over the dataset by batches.
    for batch in tqdm(data_loader): # 使用tqdm() 顯示進度條，逐批處理資料（data_loader 是tqdm()函數的 參數）
        img, _ = batch # 從batch中取出圖片，忽略原本的標籤（用 _ 表示不使用該維度的資料）

        # Forward the data
        # Using torch.no_grad() accelerates the forward process. # 加速推論
        with torch.no_grad():
            logits = model(img.to(device)) # 將圖片送到指定設備並通過模型得到logits（原始預測值）

        # Obtain the probability distributions by applying softmax on logits.
        # 將logits通過softmax轉換為機率分布
        probs = softmax(logits)

        # ---------- TODO ----------
        # Filter the data and construct a new dataset.


    # # Turn off the eval mode.
    model.train() # 切回訓練模式
    return dataset # TODO部分完成後應該回傳包含偽標籤的新資料集

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device) # new 一個 Classifier 物件
model.device = device # 在模型物件上手動添加一個屬性，儲存設備資訊

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
# # model.parameters() 表示要優化的是模型中的所有參數。
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 80

# Whether to do semi-supervised learning.
# 是否作半監督學習的開關
do_semi = False

for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if do_semi:
        # Obtain pseudo-labels for unlabeled data using trained model.
        pseudo_set = get_pseudo_labels(unlabeled_set, model) # 前面的函數

        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.
        concat_dataset = ConcatDataset([train_set, pseudo_set]) # 串接兩個 dataset
        # 如前面一樣建立 dataloader
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

    # ---------- Training ---------- 正式跑訓練迴圈
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        # 將圖片送入模型進行前向傳播，得到原始預測值（logits）
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        # 清除之前累積的梯度
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        # 反向傳播，計算各參數的梯度
        loss.backward()

        # Clip the gradient norms for stable training.
        # 梯度裁剪，防止梯度爆炸問題，將梯度範數限制在10以內
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        # 使用最佳化器更新參數
        optimizer.step()

        # Compute the accuracy for current batch.
          # logits.argmax(dim=-1): 找出預測機率最高的類別
          # 與真實標籤比較，計算正確預測的比例
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    # 驗證階段
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device)) # 驗證 loss

        # Compute the accuracy for current batch.
        # 在倒數第一個維度(最後一個)中，找出並回傳「最大值的所在 index」，即可知道預測的「類別」
        # (所以才是跟 label 比較)
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 001/080 ] loss = 2.40945, acc = 0.13781


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 001/080 ] loss = 2.46324, acc = 0.12865


  0%|          | 0/25 [00:00<?, ?it/s]

...

[ Valid | 079/080 ] loss = 1.83214, acc = 0.58177


  0%|          | 0/25 [00:00<?, ?it/s]

[ Train | 080/080 ] loss = 0.17925, acc = 0.93750


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 080/080 ] loss = 2.01846, acc = 0.58750


## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch # 此處的 label 裡面打開看是沒有東西的，因為是測試資料

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device)) # 我們只會使用 batch 回傳的 imgs，忽略 label（裡面也只是 0）

    # Take the class with greatest logit as prediction and record it.
    # 把每個 batch 的預測結果用 extend() 加總
    # .tolist() 是把型態轉換成 python list
    # 型態轉換：GPU張量 → CPU張量 → numpy陣列 → Python列表
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")